In [ ]:
import numpy as np
import pandas as pd
from gymnasium import Env, spaces
import random
from stable_baselines3 import PPO,A2C,TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
import datetime
from stable_baselines3.common.callbacks import EvalCallback,BaseCallback
from stable_baselines3.common.logger import configure
from stable_baselines3.common.evaluation import evaluate_policy
import optuna
import pickle
import torch
from pytorch_forecasting import TemporalFusionTransformer
import datetime
import warnings
import gc
from pympler import asizeof
import tracemalloc
from torch.utils.data import Dataset
warnings.filterwarnings("ignore")

In [ ]:
# 定义常量
MAX_ACTION_INT = np.int32(2000)
MAX_VALUE_INT = np.int32(9999)
MAX_VALUE_FLOAT = np.float32(999999)

In [ ]:
# 启动追踪内存使用情况
tracemalloc.start()

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

# 设置精度
torch.set_float32_matmul_precision('medium')

In [ ]:
class InventoryDataset(Dataset):
    def __init__(self):
        # 数据路径
        base_path = 'c:/OuNingyi/21级工程管理欧宁益/论文基础数据/'
        stock_filepath = base_path + 'shop_stock.csv'
        dc_stock_filepath = base_path + 'dc_stock_data.csv'
        sale_filepath = base_path + 'sale_data.csv'
        best_model_path = 'lightning_logs\\lightning_logs\\version_40\\checkpoints\\epoch=36-step=1850.ckpt'
        stock_df = pd.read_csv(stock_filepath)
        sale_df = pd.read_csv(sale_filepath)
        dc_stock_df = pd.read_csv(dc_stock_filepath)
        # 日期转换
        stock_df.dt = pd.to_datetime(stock_df.dt,format='%Y%m%d')
        sale_df['dt'] = pd.to_datetime(sale_df.ds)
        sale_df.drop(columns=['ds'],inplace=True)
        dc_stock_df.dt = pd.to_datetime(dc_stock_df.dt,format='%Y%m%d')
        # 重命名明确字段
        stock_df.columns = ['shop_code', 'goods_code', 'dt', 'shop_stock_qty']
        dc_stock_df.columns = ['dt', 'goods_code', 'dc_stock_qty', 'current_move_avg_price']
        # 取各数据集共同的最小时间
        min_dt = max(sale_df.dt.min(),stock_df.dt.min(),dc_stock_df.dt.min())
        max_dt = min(sale_df.dt.max(),stock_df.dt.max(),dc_stock_df.dt.max())
        # 读取数据格式
        main_dtypes_dict_path = base_path + 'main_dtypes_dict.json'
        with open(main_dtypes_dict_path, 'rb') as f:  
            main_dtypes_dict = pickle.load(f)  
        # 读取主数据用于深度学习模型训练
        main_dl = pd.read_csv(base_path + 'main_dl.csv', dtype=main_dtypes_dict,parse_dates=['ds'])
        main_dl.shop_code = main_dl.shop_code.astype(str).astype('category')
        main_dl.goods_code = main_dl.goods_code.astype(str).astype('category')
        main_dl.rename(columns={'ds': 'dt'}, inplace=True)

        merged_idx = main_dl.loc[main_dl.dt >= min_dt,['shop_code','goods_code','dt','time_idx']].copy()
        merged_idx.shop_code = merged_idx.shop_code.astype(int)
        merged_idx.goods_code = merged_idx.goods_code.astype(int)
        # 所有数据集的共同索引
        merged_df = merged_idx.merge(stock_df,how='left',on=['shop_code','goods_code','dt']
                                ).merge(sale_df,how='left',on=['shop_code','goods_code','dt']
                                        ).merge(dc_stock_df,how='left',on=['goods_code','dt']).fillna(0)
        # 负值处理
        merged_df.loc[merged_df.shop_stock_qty <= 0,['shop_stock_qty']] = 0
        merged_df.loc[merged_df.sale_qty <= 0,['sale_qty','sale_amt']] = 0,0
        merged_df.loc[merged_df.dc_stock_qty < 0,'dc_stock_qty'] = 0

        last7days_sale = merged_df.sort_values('dt').set_index('dt').groupby(['shop_code', 'goods_code'],as_index=False).sale_qty.rolling(window='7D').sum().reset_index()
        last7days_sale['last7day'] = last7days_sale.dt - pd.Timedelta(days=7)
        data_pred = merged_df.merge(last7days_sale.iloc[:,:-1],how='left',on=['shop_code','goods_code','dt'],suffixes=('','_last7days')).merge(
            last7days_sale.iloc[:,1:],how='left',left_on=['shop_code','goods_code','dt'],right_on=['shop_code','goods_code','last7day'],suffixes=('','_next7days'))
        # 平均补货申请满足率
        dc_supp = data_pred.groupby(['goods_code','dt','dc_stock_qty'],as_index=False).sale_qty.sum()
        dc_supp['supp_rate'] =  dc_supp.dc_stock_qty / dc_supp.sale_qty/10
        dc_supp.loc[dc_supp.supp_rate > 1, 'supp_rate'] = 1
        dc_supp_rate = dc_supp.groupby('goods_code',as_index=False).supp_rate.agg(['mean','std'])
        dc_supp_rate.columns = ['goods_code','supp_rate_mean','supp_rate_std']
        # 门店商品维度的成交均价
        avg_price_df = data_pred.groupby(['shop_code','goods_code'],as_index=False)[['sale_amt','sale_qty']].sum()
        avg_price_df['avg_sale_price'] = avg_price_df.sale_amt / avg_price_df.sale_qty
        # 商品维度的成交均价
        sku_avg_sale_df = data_pred.groupby(['goods_code'],as_index=False)[['sale_amt','sale_qty']].sum()
        sku_avg_sale_df['sku_avg_sale_price'] = sku_avg_sale_df.sale_amt / sku_avg_sale_df.sale_qty
        data_pred2 = data_pred.merge(dc_supp_rate,how='left',on='goods_code').merge(avg_price_df[['shop_code','goods_code','avg_sale_price']],how='left',on=['shop_code','goods_code']
            ).merge(sku_avg_sale_df[['goods_code','sku_avg_sale_price']],how='left',on=['goods_code'])
        data_pred2.avg_sale_price.fillna('sku_avg_sale_price',inplace=True)
        # 按每个商品的配送率，生成一个符合正态分布的随机值，作为本次实际配送数量占申请值的比例，超过1的值取1
        data_pred2['supp_random_rate'] = np.random.normal(data_pred2.supp_rate_mean.values,data_pred2.supp_rate_std.values)
        data_pred2.loc[data_pred2.supp_random_rate >1,'supp_random_rate'] = 1

        # 一年预测7天
        self.max_encoder_length = 365
        self.max_prediction_length = 7
        # 最小最大索引
        self.dt_map = data_pred2[['time_idx','dt']].drop_duplicates().set_index('time_idx')
        self.min_idx = self.dt_map[self.dt_map.dt==min_dt].index[0]
        self.max_idx = self.dt_map[self.dt_map.dt==max_dt].index[0]

        self.best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
        self.df = data_pred2
        self.predict_df = main_dl
        self.space_len = len(data_pred2[data_pred2.time_idx == self.min_idx])
        # context_length = max_encoder_length
        # prediction_length = max_prediction_length

    def __len__(self):
        return self.df.time_idx.max() - self.df.time_idx.min() +1
    
    def __dtmap__(self):
        '''日期和索引映射关系'''
        return {k:v for v,k in self.dt_map.dt.astype(str).items()}
    
    def __getitem__(self, date= None, idx = None):
        assert date is not None or idx is not None, '请输入日期或索引'
        # 确定索引的范围符合数据集的范围, 判断补货申请日期是否满足存在近一年的销售数据
        if idx is None:
            idx = self.__dtmap__()[date]
        assert self.min_idx <= idx <= self.max_idx, '索引值超出数据集范围'                
        assert idx >= self.predict_df.time_idx.min() + self.max_encoder_length, f'输入数据日期不足{self.max_encoder_length}天, 无法进行预测'

        # 提取近一年的销售数据和当天的门店库存数据
        predict_sale_data = self.predict_df[(self.predict_df.time_idx > idx - self.max_encoder_length)&(self.predict_df.time_idx <= idx + self.max_prediction_length)]
        predicted_sale = self.best_tft.predict(predict_sale_data,return_x=True,return_y=True,return_index=True)
        # # 将索引、历史数据、预测结果合并
        predict_sale_df = pd.concat([predicted_sale.index,
                                    pd.DataFrame(torch.cat([predicted_sale.x['encoder_target'],predicted_sale.output], dim=1, out=None).cpu().numpy())],axis=1)

        predict_sale_df[['shop_code', 'goods_code']] = predict_sale_df[['shop_code', 'goods_code']].apply(lambda x: x.values.astype(np.int32))
        predict_input = predict_sale_df[['shop_code','goods_code',365,366,367,368,369,370,371]]
        predict_input['future_7days_sale'] = predict_input[[365,366,367,368,369,370,371]].sum(axis=1)

        input_dl = self.df[self.df.time_idx == idx].merge(predict_input[['shop_code','goods_code','future_7days_sale']],how='inner',on=['shop_code','goods_code'])
        # del predict_sale_data, predicted_sale, predict_sale_df, predict_input
        # gc.collect()
        # torch.cuda.empty_cache()
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),f'时间索引{idx}预测完成')
        
        return {
                'shop_stock_qty':input_dl.shop_stock_qty.values.astype(np.int32),
                'dc_stock_qty': input_dl.dc_stock_qty.values.astype(np.int32),
                'avg_cost': input_dl.current_move_avg_price.values.astype(np.float32),
                'avg_price': input_dl.avg_sale_price.values.astype(np.float32),
                'future_7days_sale_qty':input_dl.future_7days_sale.values.astype(np.float32),
                'sale_qty_last7days':input_dl.sale_qty_last7days.values.astype(np.int32),
                'supp_rate_mean':input_dl.supp_rate_mean.values.astype(np.float32),
                'supp_rate_std':input_dl.supp_rate_std.values.astype(np.float32),
                'sale_qty_next7days':input_dl.sale_qty_next7days.values.astype(np.float32),        
                'supp_random_rate':input_dl.supp_random_rate.values.astype(np.float32),
                }


In [ ]:
my_dataset = InventoryDataset()

In [ ]:
my_dataset

In [ ]:
# snapshot1 = tracemalloc.take_snapshot()
# top_stats_001 = snapshot1.statistics('traceback')
# for stat in top_stats_001:
#     print(stat)

In [ ]:
# 定义库存管理环境
class InventoryManagementEnv(Env):
    def __init__(self,dataset,MAX_ACTION_INT,MAX_VALUE_INT,MAX_VALUE_FLOAT,date= None, idx = None):
        super().__init__()
        self.MAX_ACTION_INT = MAX_ACTION_INT
        self.MAX_VALUE_INT = MAX_VALUE_INT
        self.MAX_VALUE_FLOAT = MAX_VALUE_FLOAT
        self.reward = 0
        self.dataset = dataset
        self.date = date
        self.idx = idx
        self.reset()

        # 定义动作
        self.action_space = spaces.Box(
                        low=0,
                        high=self.MAX_ACTION_INT,
                        shape=(self.dataset.space_len,),
                        dtype=np.int32,)

        # 定义观测空间  
        self.observation_space = spaces.Dict(
                {
                    "shop_stock_qty": spaces.Box(
                        low=0,
                        high=self.MAX_VALUE_INT,
                        shape=(self.dataset.space_len,),
                        dtype=np.int32,
                    ),
                    "dc_stock_qty": spaces.Box(
                        low =0,
                        high=self.MAX_VALUE_FLOAT,
                        shape=(self.dataset.space_len,),
                        dtype=np.int32,
                    ),
                    "avg_cost": spaces.Box(
                        low =0,
                        high=self.MAX_VALUE_FLOAT,
                        shape=(self.dataset.space_len,),
                        dtype=np.float32,
                    ),
                    "avg_price": spaces.Box(
                        low=0,
                        high=self.MAX_VALUE_FLOAT,
                        shape=(self.dataset.space_len,),
                        dtype=np.float32,
                    ),
                    "future_7days_sale_qty": spaces.Box(
                        low =-self.MAX_VALUE_INT,
                        high=self.MAX_VALUE_INT,
                        shape=(self.dataset.space_len,),
                        dtype=np.float32,
                    ),
                    "sale_qty_last7days": spaces.Box(
                        low =-self.MAX_VALUE_INT,
                        high=self.MAX_VALUE_INT,
                        shape=(self.dataset.space_len,),
                        dtype=np.int32,
                    ),
                    "supp_rate_mean": spaces.Box(
                        low = 0 ,
                        high= 1,
                        shape=(self.dataset.space_len,),
                        dtype=np.float32,
                    ),
                    "supp_rate_std": spaces.Box(
                        low = 0 ,
                        high= 1,
                        shape=(self.dataset.space_len,),
                        dtype=np.float32,
                    ),
                })       

    
    def env_data(self, date= None, idx = None):
        env_state = self.dataset.__getitem__(date,idx)
        # 智能体可观察数据
        self.shop_stock_qty = env_state['shop_stock_qty']
        self.dc_stock_qty = env_state['dc_stock_qty']
        self.avg_cost= env_state['avg_cost']
        self.avg_price = env_state['avg_price']
        self.future_7days_sale_qty = env_state['future_7days_sale_qty']
        self.sale_qty_last7days = env_state['sale_qty_last7days']
        self.supp_rate_mean = env_state['supp_rate_mean']
        self.supp_rate_std = env_state['supp_rate_std']
        # 智能体不可观察数据, 用于评估结果
        self.sale_qty_next7days = env_state['sale_qty_next7days']      
        self.supp_random_rate = env_state['supp_random_rate']
        # del env_state
        # gc.collect()
        # torch.cuda.empty_cache()

    def reset(self,seed=None,options=None):
        if seed is not None:
            super().reset(seed=seed)
        self.reward = 0        
        # 初始化为第一天的环境状态
        if self.idx is not None:
            self.env_data(idx= self.idx)
        elif self.date is not None:
            self.env_data(date= self.date)
        else:
            self.idx = self.dataset.min_idx
            self.env_data(idx= self.idx)

        return {"shop_stock_qty": self.shop_stock_qty, 
                "dc_stock_qty": self.dc_stock_qty, 
                "avg_cost": self.avg_cost, 
                "avg_price": self.avg_price,
                "future_7days_sale_qty": self.future_7days_sale_qty,
                "sale_qty_last7days" : self.sale_qty_last7days,
                "supp_rate_mean": self.supp_rate_mean,
                "supp_rate_std": self.supp_rate_std},{}

    def step(self, action: np.ndarray):
        # 将采购订单数量乘以本次满足率做用实际配送的商品数量,四舍五入成整数
        pur_sum = np.round(action * self.supp_random_rate).astype('int32')
                    
        # 随后7天的实际销售
        sale_qty = np.where(self.shop_stock_qty + pur_sum > self.sale_qty_next7days,self.sale_qty_next7days,self.shop_stock_qty + pur_sum ).astype('int32')
        # 更新期末库存数量
        end_shop_stock_qty = self.shop_stock_qty + pur_sum - self.sale_qty_next7days
        
        # 计算奖励
        # 销售奖励（正数）
        sale_reward = sale_qty *(self.avg_price -self.avg_cost) 
        # 缺货损失（负数）       
        shortage_reward = np.where(end_shop_stock_qty <0,end_shop_stock_qty,0) *(self.avg_price -self.avg_cost)
        # 过量库存惩罚（负数）
        excess_stock_reward = np.where(self.sale_qty_next7days >0 ,np.where(end_shop_stock_qty >0,end_shop_stock_qty,0) *self.avg_cost / self.sale_qty_next7days *7 /365 *0.1,
                                       end_shop_stock_qty *self.avg_cost)

        self.reward += (sale_reward + shortage_reward + excess_stock_reward).sum()
        # 判断是否超过最大允许日期        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}, 索引{self.idx}step更新完成')

        if self.idx >= self.dataset.max_idx:
            truncated = True
        else:
            truncated = False           
            # 将环境设为对应日期的环境
            self.idx = self.idx + 1
            self.env_data(idx= self.idx)

        return {"shop_stock_qty": self.shop_stock_qty, 
                "dc_stock_qty": self.dc_stock_qty, 
                "avg_cost": self.avg_cost, 
                "avg_price": self.avg_price,
                "future_7days_sale_qty": self.future_7days_sale_qty,
                "sale_qty_last7days" : self.sale_qty_last7days,
                "supp_rate_mean": self.supp_rate_mean,
                "supp_rate_std": self.supp_rate_std}, self.reward,False,truncated,{'action':action,'end_shop_stock_qty':end_shop_stock_qty}
    
    def render(self, mode: str = "human") -> None:
        pass

    def close(self) -> None:
        pass


In [ ]:
env = InventoryManagementEnv(my_dataset,MAX_ACTION_INT,MAX_VALUE_INT,MAX_VALUE_FLOAT)

In [ ]:
# 检查环境是否符合运行要求
# check_env(env)

In [ ]:
def optimize_a2c(trial,env):
    # 创建环境
    env = Monitor(env)

    # 定义超参数空间
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    n_steps = trial.suggest_int("n_steps", 8, 128)
    ent_coef = trial.suggest_loguniform('ent_coef', 0.00000001, 0.1)
    vf_coef = trial.suggest_uniform('vf_coef', 0, 1)
    max_grad_norm = trial.suggest_uniform('max_grad_norm', 0.3, 5.0)

    model = A2C("MultiInputPolicy", env, learning_rate=learning_rate,
                n_steps=n_steps, ent_coef=ent_coef, vf_coef=vf_coef,
                max_grad_norm=max_grad_norm, tensorboard_log="./a2c_tensorboard/",
                verbose=2,device='cpu')
    model.learn(total_timesteps=1)

    # 评估模型
    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=1)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return mean_reward, std_reward 

def objective(triall,env):
    try:
        return optimize_a2c(triall,env)
    except Exception as e:
        print(f"Error: {e}")
        return 0.0  # Return a default value in case of an error    

if __name__ == "__main__":
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: optimize_a2c(trial, env), n_trials=50, n_jobs=1)
    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
snapshot2 = tracemalloc.take_snapshot()
# snapshot2.compare_to(snapshot2, 'traceback')
# top_stats_002 = snapshot.statistics('traceback')
# for stat in top_stats_002:
#     print(stat)

In [ ]:
top_stats_002 = snapshot2.statistics('traceback')
for stat in top_stats_002:
    print(stat)

In [ ]:
model = A2C("MultiInputPolicy",env, verbose=2,tensorboard_log="a2c_tensorboard",learning_rate=0.002363384178934522,gamma=0.95,n_steps=82)
# 循环训练100个回合
for episode in range(100):
    observation, _ = env.reset()
    done = False
    truncated = False

    while not (done or truncated):
        action, _states = model.predict(observation, deterministic=False)
        next_observation, reward, done, truncated, info = env.step(action)
        observation = next_observation
    if done or truncated:
        observation, _ = env.reset()    
    # 回合结束, 获得经验
    model.learn(total_timesteps=1, reset_num_timesteps=False)
# 保存模型
model.save("inventory_management_a2c")

In [ ]:
model.load("inventory_management_a2c")